# Exploratory Data Analysis

&lt; Work in progress &gt;

__TODO / MISSING__:
* Add hour of day to comments for further exploration
* Add day of week to comments
* Class distribution: use histograms instead of size()
* Scatter plots
* Violin plots


In [76]:
import numpy as np
import matplotlib.pyplot as plt
# Not necessary, but I like the ggplot style better
plt.style.use('ggplot')

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
df_art = pd.read_csv('articles_2017_08.csv')
df_com = pd.read_csv('comments_2017_08.csv')
# Make float better readable
pd.options.display.float_format = '{:.0f}'.format

### First Look

In [77]:
df_art.head(3)

,tId,article_id,updated,num_comments,link,header,sub,text
0,30600370,30600370,1502015309,12,/leben/reisen/story/Fuenf-Gruende-fuer-eine-Re...,Fünf Gründe für eine Reise nach Alabama,Sweet Home Alabama! Der Staat im Südosten der ...,"Feinste Sandstrände, kilometerlange Maisfelder..."
1,11684042,11684042,1502015306,4,/ausland/news/story/Lopez-aus-Gefaengnis-in-Ha...,Lopez aus Gefängnis in Hausarrest entlassen,Vier Tage nach seiner Festnahme befindet sich ...,Vier Tage nach seiner Festnahme ist der venezo...
2,29297244,29297244,1502015324,39,/sport/weitere/story/29297244,Buhmann Gatlin holt Gold – Bolt wird gefeiert,Usain Bolt muss auf seine 12. WM-Goldmedaille ...,(sr/fal/sda)


In [78]:
df_com.head(3)

,tId,cId,mob,vup,vdo,tit,aut,time,con
0,30284249,329_485,1,5,2,@Rebby,Störenfried,am 06.08.2017 08:59,"Wäre doch machbar, Störsender in Autos einzub..."
1,25420673,1_37,1,0,0,@T. Paul,Papa Bär,am 05.08.2017 22:02,Hier. Aber ich bin ja nur ein Papa Bär. Hüte ...
2,30284249,47_47,1,39,7,Das liebe Handy...,Heinz,am 05.08.2017 19:00,Und dazu brauchts ne Studie? Legt im allgemei...


In [79]:
print(df_art.shape)
print(df_com.shape)

(21, 8)
(1307, 9)


In [80]:
print(df_art.dtypes)
print(' ')
print(df_com.dtypes)

tId               int64
article_id        int64
updated         float64
num_comments      int64
link             object
header           object
sub              object
text             object
dtype: object
 
tId      int64
cId     object
mob      int64
vup      int64
vdo      int64
tit     object
aut     object
time    object
con     object
dtype: object


In [81]:
df_art.describe()

# Summary:
# No missing attributes since count is the same everywhere
# No negative values, chi^2 etc. can be used

,tId,article_id,updated,num_comments
count,21,21,21,21
mean,21993259,21993259,1502015317,63
std,6324191,6324191,8,95
min,11684042,11684042,1502015306,0
25%,15904700,15904700,1502015309,6
50%,22101669,22101669,1502015316,16
75%,26624359,26624359,1502015325,39
max,30715052,30715052,1502015328,296


In [82]:
df_com.describe()

# Summary:
# No missing attributes 
# No negative values

,tId,mob,vup,vdo
count,1307,1307,1307,1307
mean,19864521,1,40,19
std,6589672,0,69,36
min,11684042,0,0,0
25%,13942829,0,8,3
50%,19423348,1,22,8
75%,25420673,1,46,18
max,30715052,1,969,316


In [83]:
print(df_art.skew())
# Makes only sense on 'num_comments'

tId            -0
article_id     -0
updated         0
num_comments    2
dtype: float64


In [84]:
print(df_com.skew())
# Skew is not high enough that it has to be corrected

tId    0
cId    1
mob   -1
vup    7
vdo    4
dtype: float64


### Feature Engineering
To make further analysis possible concerning time features.

In [108]:
from datetime import datetime

def get_dt_obj(time):
    time = time.replace('am ', '')
    # Make datetime object from string
    return datetime.strptime(time, '%d.%m.%Y %H:%M')

def get_hour_of_day(time):
    return get_dt_obj(time).hour

def get_weekday(time):
    return get_dt_obj(time).weekday()

# Basically same as "the hour of week"
def get_weekday_float(time):
    hour = float(get_hour_of_day(time))
    weekday = get_weekday(time)
    return float(weekday) + hour / 24

df_com['hour'] = df_com['time'].apply(get_hour_of_day)
df_com['weekday'] = df_com['time'].apply(get_weekday) # 0 = Monday
df_com['weekday_fl'] = df_com['time'].apply(get_weekday_float)

### Class Distribution

In [86]:
# 'df_art' has no classes
# 'df_com' 
print(df_com.groupby('mob').size())
print(df_com.groupby('hour').size())
print(df_com.groupby('weekday').size())

mob
0    334
1    973
dtype: int64
hour
0      33
1      26
2      12
3       7
4      15
5      29
6      58
7      62
8      74
9      63
10     60
11     33
12     55
13     13
14     15
15     18
16     31
17     32
18     67
19    149
20    108
21     87
22    130
23    130
dtype: int64
weekday
0     82
1      2
2    102
3    311
4     84
5    300
6    426
dtype: int64


### Correlation
Let's find relations between some of the attributes. Correlation requires continuous data.

In [111]:
# Only for these columns
cols = ['hour', 'weekday', 'weekday_fl', 'mob', 'vup', 'vdo']
df = df_com[cols]

# Change back how float is displayed
pd.options.display.float_format = '{:.4f}'.format

# Get pearson co-efficients
df_corr = df.corr()

'''
# Plot heatmap
fig, ax = plt.subplots()
heatmap = ax.pcolor(df_corr, cmap=plt.cm.Blues, alpha=0.8)

# Format
fig = plt.gcf()
fig.set_size_inches(2, 2)
ax.set_frame_on(False)
ax.grid(False)

# Put the major ticks at the middle of each cell
ax.set_yticks(np.arange(df_corr.shape[0]) + 0.5, minor=False)
ax.set_xticks(np.arange(df_corr.shape[1]) + 0.5, minor=False)

# Want a more natural, table-like display
ax.invert_yaxis()
ax.xaxis.tick_top()
ax.set_xticklabels(cols, minor=False)
ax.set_yticklabels(df_corr.index, minor=False)
plt.xticks(rotation=90)

# Turn off all the ticks
ax = plt.gca()
for t in ax.xaxis.get_major_ticks():
    t.tick1On = False
    t.tick2On = False
for t in ax.yaxis.get_major_ticks():
    t.tick1On = False
    t.tick2On = False

plt.show()
# Summary:
# The only halfway interesting correlation is between 'vup' and 'vdo'
'''
df_corr

# Summary:
# Correlations are low.
# The only interesting correlation is between 'vup' and 'vdo'.

,hour,weekday,weekday_fl,mob,vup,vdo
hour,1.0000,0.1547,0.3063,0.0045,0.1775,0.1863
weekday,0.1547,1.0000,0.9879,-0.0820,0.1056,0.1118
weekday_fl,0.3063,0.9879,1.0000,-0.0783,0.1297,0.1370
mob,0.0045,-0.0820,-0.0783,1.0000,0.0309,0.0959
vup,0.1775,0.1056,0.1297,0.0309,1.0000,0.2797
vdo,0.1863,0.1118,0.1370,0.0959,0.2797,1.0000


### Scatter plot